# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [26]:
import pandas as pd
import numpy as np
from scipy import stats

In [27]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [28]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [29]:
# check b/w proportion
data.groupby('race').call.count()

race
b    2435
w    2435
Name: call, dtype: int64

In [30]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [31]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

# 1. What test is appropriate for this problem? Does CLT apply?

Two sample test. CLT applies as the sample size is large (>30) and the observations are independent of each other.

# 2. What are the null and alternate hypotheses?

Null hypothesis: The callback rate between the two groups is the same i.e. race has no impact on whether a candidate receives a callback. 
Alternate hypothesis: The callback rate between the two groups is not the same i.e. race has an impact on whether a candidate receives a callback.

In [32]:
w = data[data.race=='w']
b = data[data.race=='b']

# 3. Compute margin of error, confidence interval, and p-value.

In [33]:
# calculations
cr_b = sum(data[data.race=='b'].call) / len(data[data.race=='b'].call)
cr_w = sum(data[data.race=='w'].call) / len(data[data.race=='w'].call)

diff_means = cr_b - cr_w

print('Sample callback rate for black-sounding names: %f' % (cr_b))
print('Sample callback rate for white-sounding names: %f' % (cr_w))
print('Difference in sample callback rates: %f' % (diff_means))

Sample callback rate for black-sounding names: 0.064476
Sample callback rate for white-sounding names: 0.096509
Difference in sample callback rates: -0.032033


In [34]:
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates

call_w = data[data['race']=='w'].call
call_b = data[data['race']=='b'].call

# difference in mean
diff_means = np.mean(call_w) - np.mean(call_b)

# compute mean count
mean_count = np.mean(np.concatenate((call_w,call_b)))

# get shifted data sets
call_w_shifted = call_w - np.mean(call_w) + mean_count
call_b_shifted = call_b - np.mean(call_b) + mean_count

# get bootstrap replicates
bs_reps_call_w = draw_bs_reps(call_w_shifted, np.mean, size=10000)
bs_reps_call_b = draw_bs_reps(call_b_shifted, np.mean, size=10000)

# get replicates of difference of means
bs_replicates = bs_reps_call_w-bs_reps_call_b

# print p-value
p = np.sum(bs_replicates >= diff_means)/len(bs_replicates)
print('p-value =', p)

p-value = 0.0


In [35]:
stats.ttest_ind(call_w,call_b)

Ttest_indResult(statistic=4.1147052908617514, pvalue=3.9408021031288859e-05)

# 4. Write a story describing the statistical significance in the context or the original problem.

Since the p-value is < 0.05, we reject the null hypothesis that the callback rate between the two groups is the same. This means that there's likely a difference in callback rates between white-sounding names and black-sounding names.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No. This analysis only indicates that race has a statistically significant impact on the callback success. We will have to look into other aspects of the data (e.g. 'education', 'ofjobs', 'yearsexp' etc) before we can conclude which the most important factor in the callback success. 